In [2]:
import numpy as np
import pandas as pd
from rapidfuzz import fuzz

zeop = pd.read_csv("Adresses ZEOP Les 3 Bassins.csv", delimiter=';')

ban = pd.read_csv("Adresses BAN Les 3 Bassins.csv", delimiter=';')


ban["type_voie"]=ban["nom_afnor"].str.split(pat=" ").str[0]
ban["nom_afnor"]=ban["nom_afnor"].str.split(pat=" ").str[1:].str.join(" ").str.upper()

zeop['type_voie'].fillna(" ", inplace=True)
zeop["type_voie"] = zeop["type_voie"].str.upper()


ban.rename(columns = {'result_housenumber':'num_voie'}, inplace=True)
ban.rename(columns = {'result_street':'nom_voie'}, inplace=True)
ban['nom_voie'] = ban['nom_voie'].str.split(pat=" ").str[1:].str.join(" ").str.upper()
ban.rename(columns = {'numero':'num_voie'}, inplace=True)
ban["type_voie"]=ban["type_voie"].str.replace("CHEM", "CHEMIN", regex=False)  
ban["type_voie"]=ban["type_voie"].str.replace("CD", "CHEMIN DEPARTEMENTAL", regex=False)  
ban["type_voie"] = ban["type_voie"].str.replace("CHEMININ", "CHEMIN", regex=False)
ban["type_voie"].unique()


ban['lat'] = (
    ban['lat']
    .str.replace(" ", "", regex=True)   # Remove spaces
    .str.replace(",", ".", regex=False) # Replace commas with dots
    .astype(float)                      # Convert to float
)

ban['lon'] = (
    ban['lon']
    .str.replace(" ", "", regex=True)   # Remove spaces
    .str.replace(",", ".", regex=False) # Replace commas with dots
    .astype(float)                      # Convert to float
)


# Fonction pour effectuer la correspondance floue sur plusieurs colonnes
def multi_column_fuzzy_match(row, df_to_match, columns_to_match, scorer=fuzz.ratio):
    """
    row: une ligne du DataFrame source
    df_to_match: DataFrame cible pour la correspondance
    columns_to_match: liste des colonnes à comparer
    scorer: méthode de scoring (e.g., fuzz.ratio, fuzz.token_sort_ratio, etc.)
    """
    best_match = None
    best_score = 0
    
    for _, target_row in df_to_match.iterrows():
        try:
            # Assurez-vous que toutes les valeurs sont converties en chaînes
            scores = [
                scorer(str(row[col]), str(target_row[col])) for col in columns_to_match
            ]
            avg_score = sum(scores) / len(columns_to_match)  # Moyenne des scores
            
            if avg_score > best_score:
                best_score = avg_score
                best_match = target_row
        except Exception as e:
            print(f"Error while processing row: {e}")
            continue  # En cas d'erreur, ignorer cette itération
    
    return pd.Series({'matched_id': best_match['id'] if best_match is not None else None,
                      'match_score': best_score})

# Colonnes à comparer
columns = ['nom_voie', 'type_voie', 'num_voie' ]

# Appliquer la correspondance floue ligne par ligne
zeop[['id', 'match_score']] = zeop.apply(
    multi_column_fuzzy_match, 
    axis=1, 
    df_to_match=ban, 
    columns_to_match=columns
)


""" 
À partir d'ici, nous pouvons tester si les coordonnées géographiques de l'adresse proposée 
par le scoring correspond à celle donnée initialement.

Coordonnées à tester 
"""
""" 
x_target = 55.29095302
y_target = -21.10194837    

# Calculer la distance euclidienne pour chaque point
ban['distance'] = np.sqrt((ban['lon'] - x_target)**2 + (ban['lat'] - y_target)**2)

# Trouver la ligne avec la distance minimale
closest_row = ban.loc[ban['distance'].idxmin()]

# Récupérer le nom de la voie
closest_nom_voie = closest_row['nom_voie']

#print(f"Le nom de la voie la plus proche est : {closest_nom_voie}")
closest_nom_voie 
"""


zeop.to_csv("nettoye.csv",index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'Adresses ZEOP Les 3 Bassins.csv'

In [ ]:
zeop.rename(columns = {'matched_id':'id'}, inplace=True)

pd.merge(zeop, ban, on="id")

NameError: name 'zeop' is not defined

In [ ]:
def Convert_BAN_to_FormatZEOP(Ban_Format, Zeop_Format):
    Zeop_Adress_Empty = Zeop_Format.iloc[0:0]

    ## ON garde uniqument les varialbes du fichier BAN
    ## ATTENTION RETIRER LES TIRETS

    Ban_To_Zeop = Zeop_Adress_Empty
    #print(Zeop_Format)
    Ban_To_Zeop['x'] = Ban_Format['lon']
    Ban_To_Zeop['y'] = Ban_Format['lat']
    Ban_To_Zeop['num_voie'] = Ban_Format['num_voie']
    Ban_To_Zeop['cp_no_voie'] = Ban_Format['rep']
    Ban_To_Zeop['type_voie'] = Ban_Format['nom_voie'].str.split().str[0] # Récupérer le premier mot
    Ban_To_Zeop['nom_voie'] = Ban_Format['nom_voie'].str.split().str[1:].str.join(' ').str.upper() # Récupére tous sauf le primier mot + UPPER

    Ban_To_Zeop['imb_id'] = Ban_Format['id']

    # Fusion des anciennes données : 



    return Ban_To_Zeop

test23 = Convert_BAN_to_FormatZEOP(ban, zeop)
test23

result = test23.set_index('imb_id').combine_first(zeop.set_index('matched_id')).reset_index()
result

C:\Users\vitry\AppData\Local\Temp\ipykernel_8708\2120858332.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ban_To_Zeop['x'] = Ban_Format['lon']
C:\Users\vitry\AppData\Local\Temp\ipykernel_8708\2120858332.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ban_To_Zeop['y'] = Ban_Format['lat']
C:\Users\vitry\AppData\Local\Temp\ipykernel_8708\2120858332.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

ValueError: cannot insert imb_id, already exists

In [ ]:
merged = pd.merge(
    ban,                         # DataFrame Ban
    zeop,                        # DataFrame Zeop
    left_on="id",                # Colonne de Ban
    right_on="matched_id",       # Colonne de Zeop
    how="inner"                  # Type de jointure
)

# Conserver uniquement les colonnes de Ban mais avec les noms de Zeop
# Créez un dictionnaire pour mapper les colonnes de Ban aux colonnes de Zeop si nécessaire
column_mapping = {
    "id": "matched_id",
    "code_postal": "code_poste",
    "nom_commune": "nom_com",
    # Ajoutez d'autres colonnes correspondantes ici si besoin
}

# Renommer les colonnes selon les noms de Zeop
merged = merged.rename(columns=column_mapping)

# Conserver uniquement les colonnes de Zeop (re-nommées)
result = merged[list(column_mapping.values())]

# Exporter ou afficher le DataFrame final
print(result)
# Optionnel : Exporter le résultat en CSV
result.to_csv("merged_result.csv", index=False)

              matched_id          matched_id  code_poste  code_poste  \
0     97423_qepkdh_00003  97423_qepkdh_00003       97426       97426   
1     97423_05v6dd_00014  97423_05v6dd_00014       97426       97426   
2     97423_05v6dd_00014  97423_05v6dd_00014       97426       97426   
3     97423_akwj91_00003  97423_akwj91_00003       97426       97426   
4     97423_akwj91_00006  97423_akwj91_00006       97426       97426   
...                  ...                 ...         ...         ...   
3174    97423_1581_00020    97423_1581_00020       97426       97426   
3175    97423_1581_00020    97423_1581_00020       97426       97426   
3176    97423_0022_00004    97423_0022_00004       97426       97426   
3177    97423_0022_00006    97423_0022_00006       97426       97426   
3178    97423_0125_00023    97423_0125_00023       97426       97426   

                nom_com            nom_com  
0     Les Trois-Bassins  Les Trois-Bassins  
1     Les Trois-Bassins  Les Trois-Bassins  
